<a href="https://colab.research.google.com/github/1kaiser/Positive/blob/main/%F0%9F%98%8ETestDough%E2%9C%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
!git clone https://github.com/1kaiser/Positive.git

Cloning into 'Positive'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.


In [56]:
%cd /content/Positive

/content/Positive


# New Section

In [1]:
!npm install -g npm
!npm install puppeteer turndown
!apt-get update
!apt install chromium-chromedriver

/tools/node/bin/npx -> /tools/node/lib/node_modules/npm/bin/npx-cli.js
/tools/node/bin/npm -> /tools/node/lib/node_modules/npm/bin/npm-cli.js
+ npm@8.5.4
added 65 packages from 18 contributors, removed 292 packages and updated 142 packages in 7.643s

added 60 packages, and audited 61 packages in 14s

8 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InR

In [25]:
%%writefile index.js

'use strict';

const puppeteer = require('puppeteer');
const fs = require('fs');
var TurndownService = require('turndown');

(async() => {
  const browser = await puppeteer.launch({executablePath:"/usr/lib/chromium-browser/chromium-browser", args:['--no-sandbox']});
  const page = await browser.newPage();

    await page.goto("https://www.whatdoesitmean.com/index632.htm", {
        waitUntil: "networkidle0",
    })

    const links = await page.$eval("body > div.section1", (el) => el.childElementCount);
    console.log(links)
    let golinks = [];
    let gg
    for (let link = 12; link <= links; link++) {
        if (link == 80 | link == 445 | link == 444 | link == 812 | link == 1165 | link == 1349 | link == 1468 | link == 1470 | link == 1742 | link == 1744 | link == 1970 | link == 1972 | link == 1975) {
            link++
        } else {
            gg = ''
            gg = await page.evaluate((link) => {
                return document.querySelector("body > div > p:nth-child(" + link + ") > b").getAttribute("style");
            }, link)
            if (gg != '' && gg == "mso-bidi-font-weight:normal") {
                //console.log(gg, link)
                let title = ''
                title = await page.evaluate((link) => {
                    return document.querySelector("body > div > p:nth-child(" + link + ")").innerText
                }, link)
                let gotolink = ''
                gotolink = await page.evaluate((link) => {
                    return document.querySelector("body > div > p:nth-child(" + link + ") > b > span > a").getAttribute('href')
                }, link)
                if (title != null && gotolink != null) {
                    //    console.log(title, gotolink)

                    golinks.push({
                        Title: title.replace(/[`~!@#$%^&*()_|+\-=?;:'"’”“,.<>\{\}\[\]\\\/]/gi, '_'),
                        Link: gotolink
                    })

                }
            }
        }



    }
    // console.log(golinks[0]["Link"], golinks[0]["Title"])
    let file = '\out.txt'
    if (!fs.existsSync(file)) {
        fs.writeFileSync(file, '', function (err) {
            if (err)
                console.log(err);
        });
    }
    var darray = fs.readFileSync('\out.txt', 'utf8').split('\n');
    let file1 = "MD"
    if (!fs.existsSync(file1)) {
        fs.mkdirSync(file1, function (err) {
            if (err)
                console.log(err);
        });
    }
    for (let i = 0; i <= golinks.length; i++) {
        // console.log(golinks[i]["Link"],'\n',golinks[i]["Title"])
        if (i == 911) {
            i++
        }
        else
        {let completed = []
        if (!darray.join().includes(golinks[i]["Link"])) {
            await page.goto(golinks[i]["Link"], {
                waitUntil: "networkidle0"
            });
            let pagedetails = ''
            var turndownService = new TurndownService()

            pagedetails = await page.content();
            pagedetails = turndownService.turndown(pagedetails)
            fs.writeFile(`MD/${golinks[i]["Title"]}.md`, pagedetails, err => {
                if (err) console.log(err)
            })

            completed.push(golinks[i]["Link"].concat('\n'))
            fs.appendFileSync('out.txt', completed.toString())
        }
        console.log("going to next link " + i + "")
    }
    }


})()

Overwriting index.js


In [27]:
!node index.js

1976
going to next link 0
going to next link 1
going to next link 2
going to next link 3
going to next link 4
going to next link 5
going to next link 6
going to next link 7
going to next link 8
going to next link 9
going to next link 10
going to next link 11
going to next link 12
going to next link 13
going to next link 14
going to next link 15
going to next link 16
going to next link 17
going to next link 18
going to next link 19
going to next link 20
going to next link 21
going to next link 22
going to next link 23
going to next link 24
going to next link 25
going to next link 26
going to next link 27
going to next link 28
going to next link 29
going to next link 30
going to next link 31
going to next link 32
going to next link 33
going to next link 34
going to next link 35
going to next link 36
going to next link 37
going to next link 38
going to next link 39
going to next link 40
going to next link 41
going to next link 42
going to next link 43
going to next link 44
going to next l

#zipping

In [107]:
!zip -r /content/20220313.zip /content/Positive/MD /content/Positive/out.txt

  adding: content/Positive/MD/ (stored 0%)
  adding: content/Positive/MD/Dr_ Strangelove Takes Flight After _Deep State_ Blindsided By _Trump State_ Counterattack.md (deflated 61%)
  adding: content/Positive/MD/Biden Warns Of _Strong Response_ For Russian Nukes In Cuba—Putin Gives _10 Megaton_ Reply.md (deflated 62%)
  adding: content/Positive/MD/Democrats Explode In Rage After Trump Inspired _Church Of Holy Hell_ Spectacle Limited To Short Run.md (deflated 63%)
  adding: content/Positive/MD/Russia Kicks Out Biden Diplomats As America Nears _Dead President_ Crossroad Revolution.md (deflated 62%)
  adding: content/Positive/MD/_Scorched Earth_ Revenge Nears After Biden Delivered _Too Late We_re F__ked_ Election Warning.md (deflated 60%)
  adding: content/Positive/MD/NASA Issues Incoming Asteroid Alert As Combat Warplanes Fill Night Skies Of America.md (deflated 62%)
  adding: content/Positive/MD/Russia Readies Nukes After Pentagon Cuts Ties With CIA And Space Force Joins Top Spy Group.md